In [1]:
# Arabic plot words handle
# !pip install arabic_reshaper
# !pip install python-bidi
from __future__ import unicode_literals
import arabic_reshaper
from bidi.algorithm import get_display

import os
import collections
import pandas as pd
import matplotlib.pyplot as plt
from configs import *
# Features Extraction word2vec
from gensim.models import Word2Vec

# Dimension reduction
from sklearn.decomposition import PCA
from direction_and_file_handleing import *
from cleaning import *


In [2]:
def dir_max_all_tweets(preprocessed_direction_path_tweets):
    all_tweets_of_direction = read_direction_analysis(preprocessed_direction_path_tweets, "none")
    print("The number of tweets in our direction is: ", len(all_tweets_of_direction))
    max_len_str = max([len(i.split()) for i in all_tweets_of_direction])
    print("The greatest string of our reviews: ", max_len_str)
    all_tweets_of_direction = [i.split() for i in all_tweets_of_direction] # split each review to list of words
    print("The firt 5 tweets are: ", all_tweets_of_direction[:5])
    return all_tweets_of_direction

In [3]:
all_tweets_of_direction1 = dir_max_all_tweets(preprocessed_direction_path_tweets)
# new_direction = handle_direction_analysis(preprocessed_direction_path_tweets, '2/')
# all_tweets_of_direction2 = dir_max_all_tweets(new_direction)


The number of tweets in our direction is:  80999
The greatest string of our reviews:  63
The firt 5 tweets are:  [['منظمه', 'الصحه', 'العالميه', 'تدعو', 'الصين', 'التحقيق', 'في', 'مصدر', 'فيروس', 'كورونا', 'الجديد', 'مع', 'ظهور', 'حاله', 'اصابه', 'في', 'تايلند', 'كيف', 'اكتشاف', 'الحاله', 'الجديده', 'كيف', 'تستعد', 'لمعالجه', 'الوضع', 'وما', 'فيروس', 'كورونا', 'التفاصيل', 'في'], ['الصحه', 'العالميه', 'ظهور', 'كورونا', 'جديد', 'في', 'الصين'], ['الصحه', 'العالميه', 'ظهور', 'كورونا', 'جديد', 'في', 'الصين', 'تبوك', 'تبوك_الان', 'صدي_تبوك'], ['وضع', 'الاطفال', 'الشاشات', 'صباحا', 'يزيد', 'خطر', 'تعرضهم', 'لمشكلات', 'في', 'النطق', 'العربيه'], ['الصحه', 'العالميه', 'تحذر', 'ظهور', 'كورونا', 'جديد', 'في', 'الصين', 'والحصيله', 'الاوليه', '59', 'اصابه', 'حتي', 'اللحظه', 'رفحاء', 'عرعر', 'طريف', 'العويقيله']]


In [4]:
number_of_features = 300
window_size = 5
min_words_count = 500

In [5]:
def word_to_vec(text_list, number_of_features, window_size, min_words_count, model_bin_saved):
    '''
    Argument:
        list of strings and each string is list of words
        size = extract 50 features for each word
        window = 7 take the context of neighbour words
        min_count = 1 consider each word that even repeated 1 time
    return:
        the word2vec model
    '''
    word_to_vec_model = Word2Vec(text_list, size =number_of_features, window = window_size, min_count=min_words_count, sg = 1) 
    print("Our word2vec model: ", word_to_vec_model)
    print("The number of frequent words of our data: ", len(word_to_vec_model.wv.vocab)) # the frequent words
    # save model
    word_to_vec_model.save('../ml_models/models_weights/word2vec/' + model_bin_saved)

    # load model
    word_to_vec_model = Word2Vec.load('../ml_models/models_weights/word2vec/'+ model_bin_saved)
    return word_to_vec_model


In [6]:
def save_words(url, word_to_vec_model):

    w2c = dict()
    for item in word_to_vec_model.wv.vocab:
        if not item.isdigit() and len(item) > 2 and not "_" in item and not (item[0] >= 'a' and item[0] <= 'z'):
            w2c[item]=word_to_vec_model.wv.vocab[item].count
    dicts = {'word2vec_model_words': list(w2c.values()), 'word2vec_model_frequency': list(w2c.keys())}
    df = pd.DataFrame(dicts)
    url = url + 'csv/word2vec_model_words.csv'
    df.to_csv(url, index=False)
    return True

In [8]:
# # direction 1
# word_to_vec_model = word_to_vec(all_tweets_of_direction1, number_of_features, 
#                                 window_size, min_words_count, "word2vec_anaysis_file_for_tablue.bin")
# _ = save_words(analysis_direction_path_tweets, word_to_vec_model)
word_to_vec_model = Word2Vec.load('../ml_models/models_weights/word2vec/word2vec_anaysis_file_for_tablue.bin' )

In [9]:
df = pd.read_csv(analysis_direction_path_tweets+ 'csv/word2vec_model_words.csv')
df.head() 

,word2vec_model_words,word2vec_model_frequency
0,2687,منظمه
1,10540,الصحه
2,6232,العالميه
3,2947,تدعو
4,213289,الصين


In [10]:
print(word_to_vec_model.wv.most_similar('الصحه'))
print(word_to_vec_model.wv.most_similar('بالفيروس'))
print(print("Feature of some words\n", word_to_vec_model['الصحه']))

[('وزاره', 0.5062141418457031), ('الصحيه', 0.4807060956954956), ('حاله', 0.4573962688446045), ('منظمه', 0.4357641041278839), ('الاجراءات', 0.41532406210899353), ('صحه', 0.4090246558189392), ('العالميه', 0.4052422046661377), ('طوارء', 0.40076807141304016), ('مصابه', 0.39856642484664917), ('اصابه', 0.394173264503479)]
[('بفيروس', 0.5231550931930542), ('فيروس_كورونا_الجديد', 0.41551750898361206), ('حاله', 0.39330053329467773), ('اصابه', 0.39247965812683105), ('الاصابه', 0.3906317353248596), ('تسجيل', 0.3867267370223999), ('الفيروس', 0.3818226456642151), ('فيروس_كورونا', 0.37590306997299194), ('المصابين', 0.36702844500541687), ('الغامض', 0.36474013328552246)]
Feature of some words
 [ 7.60068536e-01 -1.16706304e-01 -3.50713521e-01 -5.94351292e-02
 -2.05351949e-01  6.97977021e-02 -3.75328422e-01 -2.43842110e-01
 -4.94518736e-03  5.45679852e-02  1.18231945e-01  2.37496406e-01
 -2.40674689e-01  3.87593418e-01 -7.45591298e-02 -2.71199569e-02
  8.73693004e-02 -5.14979586e-02  4.71932028e-04 -2.5

/home/abdelrahman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# retrieve frequent words
words = list(word_to_vec_model.wv.vocab)

# I took just 100 words to present with PCA
words = words[:200]

# get the features of those 100 word
word_features = word_to_vec_model[words] 

# it will be 100 word * 50 which 50 feature of each word
print("The shape of our word_features is:", word_features.shape)

# Dimension reduction to display so instead of 50 dimension of each word 
#reduce to 2 dimensional that have fine display
# from 50 features to just 2 features for displaying how words related words to gether
pca = PCA(n_components=2)

# fit the model
xy_words = pca.fit_transform(word_features)

# set the visualization plot
plt.style.use(['Solarize_Light2'])
plt.figure(figsize=(12,10))

# after reduction for all features in first column as x and all features from second column as y
plt.scatter(xy_words[:, 0], xy_words[:, 1], marker='P',s=20, c="white") 

for i, word in enumerate(words):
    # handle Arabic words to display from right to left and as complete word not just separate chars
    word = arabic_reshaper.reshape(word) # handle arabic words on ploting
    word = get_display(word)
    # plot each word beside its point
    plt.annotate(word, xy=(xy_words[i, 0], xy_words[i, 1]))
plt.savefig('test.png')

/home/abdelrahman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


The shape of our word_features is: (200, 300)
